# // PART 1 //

In [105]:
get_ipython().system(u' pip install beautifulsoup4')

In [106]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [107]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df1 = pd.read_html(str(table))
df=pd.DataFrame(df1[0])

In [108]:
df=df[df.Borough != 'Not assigned']
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [109]:
idx_to_change = df.loc[df['Neighbourhood'] == "Not assigned"].index
for i in idx_to_change:
    df.iloc[i, 2] = df.iloc[i, 1] 
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [110]:
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
df=df.sample(frac=1).reset_index()
df['Neighbourhood']=df['Neighbourhood'].str.join(',')
df.head()

,Postcode,Borough,Neighbourhood
0,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."
1,M1R,Scarborough,"Maryvale,Wexford"
2,M6N,York,"The Junction North,Runnymede"
3,M6R,West Toronto,"Parkdale,Roncesvalles"
4,M4H,East York,Thorncliffe Park


In [111]:
df.shape

(103, 3)

# // PART 2 //

In [112]:
# The code was removed by Watson Studio for sharing.

In [113]:
df_data_1 = pd.read_csv(body)
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [114]:
df3=pd.merge(df, df_data_1, left_on="Postcode", right_on="Postal Code").drop('Postal Code', axis=1)
df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
1,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849
2,M6N,York,"The Junction North,Runnymede",43.673185,-79.487262
3,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325
4,M4H,East York,Thorncliffe Park,43.705369,-79.349372


# // PART 3 //

In [115]:
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Create a map of New York with neighborhoods superimposed on top.

In [116]:
address = 'toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [117]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### Work with Boroughs that contain Toronto

In [118]:
toronto_data = df3[df3['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325
1,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752
2,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763
3,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636


In [119]:
# create map of Toronto using latitude and longitude values
map_Toronto1 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto1)  
    
map_Toronto1

In [120]:
CLIENT_ID = 'O0QZED1VIJAAVEOB55E11KG0FNWTMYWST0LW4LJQK2LO0FRK' # your Foursquare ID
CLIENT_SECRET = 'XG4MOC51UF3DHGPAY1ETCH13BGSREUORD2V3SRBNOTKMARAR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O0QZED1VIJAAVEOB55E11KG0FNWTMYWST0LW4LJQK2LO0FRK
CLIENT_SECRET:XG4MOC51UF3DHGPAY1ETCH13BGSREUORD2V3SRBNOTKMARAR


In [121]:
toronto_data.loc[0, 'Neighbourhood']

'Parkdale,Roncesvalles'

In [122]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [123]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Parkdale,Roncesvalles
Harbourfront East,Toronto Islands,Union Station
High Park,The Junction South
Moore Park,Summerhill East
Harbourfront
Church and Wellesley
Harbord,University of Toronto
Design Exchange,Toronto Dominion Centre
St. James Town
Stn A PO Boxes 25 The Esplanade
Davisville North
The Danforth West,Riverdale
Davisville
North Toronto West
Queen's Park
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Business Reply Mail Processing Centre 969 Eastern
Brockton,Exhibition Place,Parkdale Village
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Runnymede,Swansea
Ryerson,Garden District
The Annex,North Midtown,Yorkville
Central Bay Street
Berczy Park
Studio District
Little Portugal,Trinity
Commerce Court,Victoria Hotel
Chinatown,Grange Park,Kensington Market
Christie
Dovercourt Village,Dufferin
The Beaches West,India Bazaar
Forest Hill North,Forest Hill West
Lawrence Park
Roselawn
Rosedale
Cabbagetown,St. James Town
A

In [124]:
print(toronto_venues.shape)
toronto_venues.head()

(1720, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Parkdale,Roncesvalles",43.64896,-79.456325,Offleash Dog Trail - High Park,43.645485,-79.458747,Dog Run
1,"Parkdale,Roncesvalles",43.64896,-79.456325,La Cubana,43.650912,-79.450909,Cuban Restaurant
2,"Parkdale,Roncesvalles",43.64896,-79.456325,The Chocolateria,43.649928,-79.450437,Dessert Shop
3,"Parkdale,Roncesvalles",43.64896,-79.456325,Revue Cinema,43.651112,-79.450961,Movie Theater
4,"Parkdale,Roncesvalles",43.64896,-79.456325,Inter Steer,43.649796,-79.450310,Eastern European Restaurant


In [125]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton,Exhibition Place,Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,14,14,14,14,14,14
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",17,17,17,17,17,17
"Cabbagetown,St. James Town",47,47,47,47,47,47
Central Bay Street,88,88,88,88,88,88
"Chinatown,Grange Park,Kensington Market",86,86,86,86,86,86
Christie,18,18,18,18,18,18


In [126]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


In [127]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
toronto_onehot.shape

(1720, 232)

In [129]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011364,0.000000,...,0.00000,0.000000,0.00000,0.00,0.011364,0.000000,0.000000,0.011364,0.000000,0.00
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.046512,0.000000,0.058140,0.011628,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.011765,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.011765,0.000000,0.011765,0.00


In [130]:
toronto_grouped.shape

(39, 232)

In [131]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2            Beer Bar  0.03
3  Seafood Restaurant  0.03
4      Farmers Market  0.03


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0     Coffee Shop  0.10
1  Breakfast Spot  0.10
2            Café  0.10
3   Grocery Store  0.05
4   Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0         Yoga Studio  0.07
1       Auto Workshop  0.07
2                 Spa  0.07
3          Skate Park  0.07
4  Light Rail Station  0.07


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Term

In [132]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [140]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,Bar,Café,Hotel,Cosmetics Shop,Restaurant,Thai Restaurant,Asian Restaurant,Concert Hall
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Steakhouse,Beer Bar,Seafood Restaurant,Café,Cheese Shop,Breakfast Spot
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Breakfast Spot,Café,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Smoke Shop,Light Rail Station,Spa,Farmers Market,Fast Food Restaurant,Brewery,Burrito Place,Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Boutique


// CLUSTER NEIGHBORHOOD //

In [141]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [142]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325,0,Gift Shop,Cuban Restaurant,Dog Run,Restaurant,Bookstore,Dessert Shop,Movie Theater,Italian Restaurant,Bar,Eastern European Restaurant
1,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Italian Restaurant,Hotel,Café,Scenic Lookout,Brewery,Fried Chicken Joint,Restaurant,History Museum
2,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763,0,Thai Restaurant,Bar,Mexican Restaurant,Café,Cajun / Creole Restaurant,Flea Market,Bakery,Speakeasy,Fried Chicken Joint,Italian Restaurant
3,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,4,Park,Playground,Trail,Summer Camp,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Farmers Market,Event Space


In [144]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters